# <b> Data Processing Practice with PySpark and SparkSQL

## Nhập về thư viện cần thiết và thực hiện cấu hình/điều chỉnh cho phù hợp với máy

### Nhập thư viện

In [93]:
#cái này là để nó không cảnh báo khi sử dụng pyspark.pandas, mà thư viện phụ thuộc vào pyarrow nên cần set về 1 cho nó đỡ càm ràm
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [265]:
#import mấy thư viện cần thiết
from pyspark.sql import SparkSession
import pyspark.pandas as ps
from pyspark.sql.functions import col, isnan, when, count, desc, mean
import pandas as pd

### Cấu hình

In [98]:
#cấu hình Spark cho phù hợp với máy tính (ps: máy tính công ty nên yếu si lýnh)
spark = (SparkSession.builder
         .config("spark.driver.memory", "4g")
         .config("spark.sql.execution.arrow.maxRecordsPerBatch", "50000")
         .config("spark.sql.shuffle.partitions", "8")
         .config("spark.network.timeout", "600s")
         .getOrCreate()
)

## Đọc file .csv

In [101]:
#mới nhảy qua chơi món Spark nên chưa quen việc thêm .show() đằng sau lắm; cái inferSchema cũng quan trọng nha, 
#cái này sẽ phải đọc thêm về cách Spark nó xử lý, cơ mà đọc trên stackoverflow, hiểu đơn giản là không thêm vào 
#nó bị thành hết String tại Spark lười
df = spark.read.csv('/Users/Alpha/datasets/Churn_Modelling.csv', inferSchema = True, header = True)
df.show()

+---------+----------+---------+-----------+---------+------+-----+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|  Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+-----+------+---------+-------------+---------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42.0|     2|      0.0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41.0|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42.0|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39.0|     1|      0.0|            2|        0|          

## Làm sạch dữ liệu

### 1. Kiểm tra Schema, Casting và đổi tên cột cho dễ hiểu

In [105]:
#check Schema của các cột trong dữ liệu, có vẻ các cột đều khá oke
df.printSchema()

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



Ở đây hầu như cột nào tên cũng bắt đầu bằng chữ viết hoa, để nguyên thì khéo nó loạn nhà luôn.

Ngoải ra thì cột 'Age' đang bị sai kiểu dữ liệu, nhiều cột đọc xong chưa hiểu ngay nó là cái gì, ví dụ như <b> 'CrCard' </b> thì khá khó hiểu ra là Credit Card nên cần đổi tên kha khá.

In [109]:
#thực hiện chuyển kiểu dữ liệu và đổi tên các cột để đọc dễ hiểu hơn
bank_churned_casting = df.select(
    col('CustomerId').cast('integer').alias('customer_id')
    , col('Surname').cast('string').alias('surname')
    , col('CreditScore').cast('integer').alias('credit_score')
    , col('Geography').cast('string').alias('home_country')
    , col('Gender').cast('string').alias('gender')
    , col('Age').cast('integer').alias('age')
    , col('Tenure').cast('integer').alias('years_stayed')
    , col('Balance').cast('double').alias('bank_balance')
    , col('NumOfProducts').cast('integer').alias('no_products_used')
    , col('HasCrCard').cast('integer').alias('hasCreditCard')
    , col('IsActiveMember').cast('integer').alias('isActive')
    , col('EstimatedSalary').cast('double').alias('salary')
    , col('Exited').cast('integer').alias('isChurned')
)

In [111]:
#thành quả
bank_churned_casting.show()

+-----------+---------+------------+------------+------+----+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|  surname|credit_score|home_country|gender| age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+---------+------------+------------+------+----+------------+------------+----------------+-------------+--------+---------+---------+
|   15634602| Hargrave|         619|      France|Female|  42|           2|         0.0|               1|            1|       1|101348.88|        1|
|   15647311|     Hill|         608|       Spain|Female|  41|           1|    83807.86|               1|            0|       1|112542.58|        0|
|   15619304|     Onio|         502|      France|Female|  42|           8|    159660.8|               3|            1|       0|113931.57|        1|
|   15701354|     Boni|         699|      France|Female|  39|           1|         0.0|               2|        

In [113]:
#check lại Schema
bank_churned_casting.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- credit_score: integer (nullable = true)
 |-- home_country: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- years_stayed: integer (nullable = true)
 |-- bank_balance: double (nullable = true)
 |-- no_products_used: integer (nullable = true)
 |-- hasCreditCard: integer (nullable = true)
 |-- isActive: integer (nullable = true)
 |-- salary: double (nullable = true)
 |-- isChurned: integer (nullable = true)



### 2. Kiểm tra trùng lặp

Mỗi khách hàng thì chỉ có một id thôi, khó mà có 2 khách đồng quy được. Thế nên là bỏ lặp đi để data nó đẹp hơn.

In [117]:
#nếu dùng sql nhiều rồi thì bước này cũng ez, cơ mà giải thích thì là đếm theo các dòng (bên dưới ghi columns hem, thực sự nó là
#group theo giá trị của cột đó, chứ đừng hiểu là gộp thành hết một cột rồi đếm), và xem dòng nào xuất hiện nhiều hơn một lần thôi
bank_churned_duplicates = bank_churned_casting.groupBy(bank_churned_casting.columns).count().filter("count > 1")

#đoạn này thì xem những id nào đang là id bị lặp ấy, thì mấy id xuất hiện nhiều hơn 1 là đang lặp rồi
#cái note là đối với kiểu dữ liệu columns trong spark thì cần collect() để nhận dữ liệu
bank_churned_duplicates_id = bank_churned_duplicates.select('customer_id').collect()
bank_churned_duplicates_id

[Row(customer_id=15682355), Row(customer_id=15628319)]

In [119]:
#giờ check dòng có id bị lặp xem máy nó bị dở hơi hay lặp thật
bank_churned_casting.filter(bank_churned_casting['customer_id'].isin(bank_churned_duplicates_list)).show()

+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+--------+---------+
|customer_id|  surname|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|  salary|isChurned|
+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+--------+---------+
|   15682355|Sabbatini|         772|     Germany|  Male| 42|           3|    75075.31|               2|            1|       0|92888.52|        1|
|   15682355|Sabbatini|         772|     Germany|  Male| 42|           3|    75075.31|               2|            1|       0|92888.52|        1|
|   15628319|   Walker|         792|      France|Female| 28|           4|   130142.79|               1|            1|       0|38190.78|        0|
|   15628319|   Walker|         792|      France|Female| 28|           4|   130142.79|               1|            1|       

Lặp thật.

In [122]:
#cơ mà mấy cái id đang bị vướng, nó ở dạng list rồi cơ mà chưa lấy ra được
#đoạn này thì nên hiểu là nếu khai báo thứ tự dòng xong chain nó với tên cột thì sẽ ra giá trị bên trong
#vd, cop dòng dưới này sang một cell khác rồi chạy thử nhé
# bank_churned_duplicates_id[0].customer_id ==> kết quả: 15682355 

#đoạn này chạy để nó ra cái mình cần thôi, báo là row để chạy lặp cho dễ hiểu thôi chứ thực chất cứ hiểu là 
#lấy giá trị từ list ra, mà cái này chạy vòng lặp python cơ bản rồi =)))
bank_churned_duplicates_list = [int(row.customer_id) for row in bank_churned_duplicates_id]

#đoạn này thì là list rồi có số má ngon rồi
bank_churned_duplicates_list

[15682355, 15628319]

In [124]:
#drop test thử xem còn bị lặp giá trị không nhá
test = bank_churned_casting.drop_duplicates()
test.show()

+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|  surname|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|   15701354|     Boni|         699|      France|Female| 39|           1|         0.0|               2|            0|       0| 93826.63|        0|
|   15643966|  Goforth|         616|     Germany|  Male| 45|           3|   143129.41|               2|            0|       1| 64327.26|        0|
|   15729599|  Lorenzo|         804|       Spain|  Male| 33|           7|     76548.6|               1|            0|       1| 98453.45|        0|
|   15804771|Velazquez|         614|      France|  Male| 51|           4|    40685.92|               1|            1| 

In [126]:
#check lại 2 con id khách hàng bị lặp ban nãy
test.filter(bank_churned_casting['customer_id'].isin(bank_churned_duplicates_list)).show()

+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+--------+---------+
|customer_id|  surname|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|  salary|isChurned|
+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+--------+---------+
|   15682355|Sabbatini|         772|     Germany|  Male| 42|           3|    75075.31|               2|            1|       0|92888.52|        1|
|   15628319|   Walker|         792|      France|Female| 28|           4|   130142.79|               1|            1|       0|38190.78|        0|
+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+--------+---------+



Thế là hết lặp rồi, không cần test nữa cho thẳng vô bước kết quả thôi.

In [202]:
#nhận kết quả
bank_churned_deduplicating = bank_churned_casting.drop_duplicates() 
bank_churned_deduplicating.filter(bank_churned_casting['customer_id'].isin(bank_churned_duplicates_list)).show()

+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+--------+---------+
|customer_id|  surname|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|  salary|isChurned|
+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+--------+---------+
|   15682355|Sabbatini|         772|     Germany|  Male| 42|           3|    75075.31|               2|            1|       0|92888.52|        1|
|   15628319|   Walker|         792|      France|Female| 28|           4|   130142.79|               1|            1|       0|38190.78|        0|
+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+--------+---------+



In [204]:
bank_churned_deduplicating.show()

+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|  surname|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|   15701354|     Boni|         699|      France|Female| 39|           1|         0.0|               2|            0|       0| 93826.63|        0|
|   15643966|  Goforth|         616|     Germany|  Male| 45|           3|   143129.41|               2|            0|       1| 64327.26|        0|
|   15729599|  Lorenzo|         804|       Spain|  Male| 33|           7|     76548.6|               1|            0|       1| 98453.45|        0|
|   15804771|Velazquez|         614|      France|  Male| 51|           4|    40685.92|               1|            1| 

oke ngon rồi đấy!

### 3. Kiểm tra giá trị null từng cột

In [194]:
#show tên cột cho xanh chín
df_column = bank_churned_deduplicating.columns
df_column

['customer_id',
 'surname',
 'credit_score',
 'home_country',
 'gender',
 'age',
 'years_stayed',
 'bank_balance',
 'no_products_used',
 'hasCreditCard',
 'isActive',
 'salary',
 'isChurned']

Ý tưởng cho việc kiểm tra null là đếm số giá trị null của mỗi cột. Nếu pyspark nó ngon như pandas thì phần này làm khá dễ, 
chỉ cần df.info() là xong rồi, cơ mà pyspark nó dễ bị lỗi lắm. Nếu chạy kiểu đó, pyspark nó sẽ kêu lỗi tại méo xử lý được
nên là nếu muốn đếm null thì phải thủ công, nghĩ theo hướng SQL xíu.

Giờ chạy con vòng lặp các cột rồi đếm nhá.

In [206]:
temp_null_container = bank_churned_deduplicating.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_column]).show()
#Tạo một container để xem cả cái bảng nó null ở đâu
temp_null_container

+-----------+-------+------------+------------+------+---+------------+------------+----------------+-------------+--------+------+---------+
|customer_id|surname|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|salary|isChurned|
+-----------+-------+------------+------------+------+---+------------+------------+----------------+-------------+--------+------+---------+
|          0|      0|           0|           1|     0|  1|           0|           0|               0|            1|       1|     0|        0|
+-----------+-------+------------+------------+------+---+------------+------------+----------------+-------------+--------+------+---------+



Từ kết quả trên thì thấy các cột này bị null này: <b>'home_country', 'age', 'hasCreditCard', 'isActive'

Giờ mình xử lý từng cột một thôi là ngon rồi.

#### 3.1. Cột 'home_country'

In [208]:
bank_churned_deduplicating.select(col('home_country')).distinct().show()

+------------+
|home_country|
+------------+
|       Spain|
|      France|
|     Germany|
|        NULL|
+------------+



Đầu tiên xem có những giá trị nào trong cột, thì ta thấy một con NULL to đùng, cơ mà theo bảng trên thì có mỗi một giá trị NULL thôi.
Vẫn cần sửa để dataset nó đẹp xíu, thì mình thay NULL thành Unknown là xong.

In [226]:
test = bank_churned_deduplicating.fillna(value='Unknown', subset=['home_country'])
test.show()

+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|  surname|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|   15701354|     Boni|         699|      France|Female| 39|           1|         0.0|               2|            0|       0| 93826.63|        0|
|   15643966|  Goforth|         616|     Germany|  Male| 45|           3|   143129.41|               2|            0|       1| 64327.26|        0|
|   15729599|  Lorenzo|         804|       Spain|  Male| 33|           7|     76548.6|               1|            0|       1| 98453.45|        0|
|   15804771|Velazquez|         614|      France|  Male| 51|           4|    40685.92|               1|            1| 

In [228]:
test.select(col('home_country')).distinct().show()

+------------+
|home_country|
+------------+
|       Spain|
|     Unknown|
|      France|
|     Germany|
+------------+



Này là cái cột cắn giá trị 'Unknown' rồi, lát mình gán vào dữ liệu cần thôi.

Tại sao là lát í hả, ấy là vì mình có thể fillna dưới dạng dict, tức là fill giá trị null của từng cột với một giá trị riêng mình khoái.

Thế nên xử lý từng cột một thôi!

#### 3.2. Cột 'age'

In [260]:
#distinct luôn để xem tìm Null có dễ như lần trước không
test_age = bank_churned_deduplicating.select(col('age')).distinct()

#có vẻ là không rồi =)))
test_age.show()

+---+
|age|
+---+
| 38|
| 46|
| 67|
| 70|
| 18|
| 39|
| 51|
| 48|
| 62|
| 23|
| 64|
| 82|
| 74|
| 31|
| 58|
| 49|
| 57|
| 76|
| 77|
| 41|
+---+
only showing top 20 rows



In [262]:
#sort lại cột để nhìn NULL
test_age.sort(col('age')).show()

+----+
| age|
+----+
|NULL|
|  18|
|  19|
|  20|
|  21|
|  22|
|  23|
|  24|
|  25|
|  26|
|  27|
|  28|
|  29|
|  30|
|  31|
|  32|
|  33|
|  34|
|  35|
|  36|
+----+
only showing top 20 rows



May là toàn 1 NULL hết nên cũng chill =)) giờ tính trung bình cột age rồi thay giá trị hiện tại vào là oke

In [289]:
test_age.select(mean(col('age'))).collect()

[Row(avg(age)=52.6)]

Ca này thì thôi khỏi chạy list làm gì cho màu mè =))) replace NULL với 52.6 là oke

#### 3.3. Cột 'hasCreditCard', 'isActive'

2 cột này thì xử lý y hệt nhau luôn, tại nó đều là cột boolean nên mình thay thành một cái Unknown riêng cho nó, nên cho nó là 2 cả đôi.

#### 3.4. Fill NULL

In [329]:
test = bank_churned_deduplicating.fillna({'home_country':'Unknown'
                                   , 'age': 53 #lẽ ra là 52.6 cơ mà làm tròn lên tại cột này đang là integer
                                   , 'hasCreditCard' : 2
                                   , 'isActive': 2
                                  })
test.show()

+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|  surname|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|   15701354|     Boni|         699|      France|Female| 39|           1|         0.0|               2|            0|       0| 93826.63|        0|
|   15643966|  Goforth|         616|     Germany|  Male| 45|           3|   143129.41|               2|            0|       1| 64327.26|        0|
|   15729599|  Lorenzo|         804|       Spain|  Male| 33|           7|     76548.6|               1|            0|       1| 98453.45|        0|
|   15804771|Velazquez|         614|      France|  Male| 51|           4|    40685.92|               1|            1| 

In [330]:
#check xem còn ông nào null không
test.select([
    count(when(isnan(c) | col(c).isNull(), c)).alias(c)
    for c in test.columns
]).show()

+-----------+-------+------------+------------+------+---+------------+------------+----------------+-------------+--------+------+---------+
|customer_id|surname|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|salary|isChurned|
+-----------+-------+------------+------------+------+---+------------+------------+----------------+-------------+--------+------+---------+
|          0|      0|           0|           0|     0|  0|           0|           0|               0|            0|       0|     0|        0|
+-----------+-------+------------+------------+------+---+------------+------------+----------------+-------------+--------+------+---------+



Có vẻ là hết null òi, ngon đét nhể. Giờ mình gán lại vào cái cục biến ban đầu là oke.

In [334]:
bank_churned_null_removed = bank_churned_deduplicating.fillna({'home_country':'Unknown'
                                   , 'age': 53
                                   , 'hasCreditCard' : 2
                                   , 'isActive': 2
                                  })
bank_churned_null_removed.show()

+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|  surname|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|   15701354|     Boni|         699|      France|Female| 39|           1|         0.0|               2|            0|       0| 93826.63|        0|
|   15643966|  Goforth|         616|     Germany|  Male| 45|           3|   143129.41|               2|            0|       1| 64327.26|        0|
|   15729599|  Lorenzo|         804|       Spain|  Male| 33|           7|     76548.6|               1|            0|       1| 98453.45|        0|
|   15804771|Velazquez|         614|      France|  Male| 51|           4|    40685.92|               1|            1| 

In [336]:
bank_churned_null_removed.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- credit_score: integer (nullable = true)
 |-- home_country: string (nullable = false)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = false)
 |-- years_stayed: integer (nullable = true)
 |-- bank_balance: double (nullable = true)
 |-- no_products_used: integer (nullable = true)
 |-- hasCreditCard: integer (nullable = false)
 |-- isActive: integer (nullable = false)
 |-- salary: double (nullable = true)
 |-- isChurned: integer (nullable = true)



### 4. Kiểm tra từng cột một

Thế tại sao tự nhiên đi check từng cột một? Sau khi mà mình thay thiếc các thứ ấy, thì cái vấn đề là vẫn có thể có mấy dữ liệu 
nó bị dở người, kiểu lọt số 0 hay lọt cái ký tự lạ vô. Nên trước mắt là check distinct với cả xuôi ngược.

#### 4.1. 'customer_id'

In [368]:
bank_churned_null_removed.orderBy('customer_id', ascending = False).show()

+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|  surname|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+---------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|   15815690| Akabueze|         614|       Spain|Female| 40|           3|    113348.5|               1|            1|       1| 77789.01|        0|
|   15815660|    Mazzi|         758|      France|Female| 34|           1|   154139.45|               1|            1|       1| 60728.89|        0|
|   15815656|  Hopkins|         541|     Germany|Female| 39|           9|   100116.67|               1|            1|       1| 199808.1|        1|
|   15815645|   Akhtar|         481|      France|  Male| 37|           8|   152303.66|               2|            1| 

In [366]:
bank_churned_null_removed.orderBy('customer_id', ascending = True).show()

+-----------+----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|   surname|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|   15565701|     Ferri|         698|       Spain|Female| 39|           9|   161993.89|               1|            0|       0| 90212.38|        0|
|   15565706|  Akobundu|         612|       Spain|  Male| 35|           1|         0.0|               1|            1|       1| 83256.26|        1|
|   15565714|  Cattaneo|         601|      France|  Male| 47|           1|    64430.06|               2|            0|       1| 96517.97|        0|
|   15565779|      Kent|         627|     Germany|Female| 30|           6|    57809.32|               1|        

Cột này toàn số thôi nên cứ bọc đầu bọc đuôi không có vấn đề gì là ngon thôi.

#### 4.2. 'surname'

Nói một chút về ý nghĩa cột này với cột 'customer_id' nhé. Dataset của mình có thể dùng cho 2 mục đích: 
- <b>1: Churn Prediction</b>, lúc này thì các cột chuyển về dạng số hết, cột này và 'customer_id' là 2 cột thừa tại nó không có ý nghĩa gì cả.
<b>vd:</b> nếu bạn so sánh mối tương quan, chả nhẽ nếu tên bắt đầu là 'Z' thì sẽ đi vay ngân hàng nhiều hơn à =))
hay nếu khách hàng nào có id càng to càng vay lắm. Đủ để thấy 2 cột này vô giá trị nếu đưa vào học máy. 
- <b>2: Customer Churn Analysis, Customer Service...</b>, mấy cái này thiên về phân tích mô tả, lúc này mình có thể cần tới customer_id
để tìm ra khách hàng vay ác nhất trong năm vừa rồi để trao thưởng cho họ, hoặc làm đủ thứ khác liên quan tới CSKH. Cơ mà mình cần tên không, hay chỉ cần đưa id cho bên CSKH/Sale là họ tự tìm được khách. Dĩ nhiên là đưa id rồi, vậy nên cột 'surname' không có ý nghĩa gì hết.

Từ 2 cái này, mình có thể đi đến kết luận là <b>drop luôn cột 'surname' đi cho đỡ tốn dữ liệu =)))

In [400]:
#đặt tên là optimized vì mình bỏ cột thừa thãi
bank_churned_optimized = bank_churned_null_removed.drop('surname')
bank_churned_optimized.show()

+-----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|   15701354|         699|      France|Female| 39|           1|         0.0|               2|            0|       0| 93826.63|        0|
|   15643966|         616|     Germany|  Male| 45|           3|   143129.41|               2|            0|       1| 64327.26|        0|
|   15729599|         804|       Spain|  Male| 33|           7|     76548.6|               1|            0|       1| 98453.45|        0|
|   15804771|         614|      France|  Male| 51|           4|    40685.92|               1|            1|       1| 46775.28|        0|
|   15651280|         742|     Germany|  

#### 4.3. 'credit_score'

In [407]:
bank_churned_optimized.orderBy('credit_score', ascending = False).show()

+-----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|   15649124|         850|      France|  Male| 30|           9|   121535.18|               1|            0|       0| 40313.47|        0|
|   15700801|         850|     Germany|  Male| 42|           6|    84445.68|               3|            0|       1| 60021.34|        1|
|   15786078|         850|      France|Female| 28|           9|         0.0|               2|            1|       0|185821.41|        0|
|   15716334|         850|       Spain|Female| 45|           2|   122311.21|               1|            1|       1|  19482.5|        0|
|   15662641|         850|      France|  

In [409]:
bank_churned_optimized.orderBy('credit_score', ascending = True).show()

+-----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|   15765173|         350|      France|Female| 60|           3|         0.0|               1|            0|       0|113796.15|        1|
|   15668309|         350|      France|Female| 40|           0|   111098.85|               1|            1|       1|172321.21|        1|
|   15758813|         350|     Germany|  Male| 39|           0|    109733.2|               2|            0|       0|123602.11|        1|
|   15685372|         350|       Spain|  Male| 54|           1|   152677.48|               1|            1|       1|191973.49|        1|
|   15803202|         350|      France|  

Điểm tín dụng trông có vẻ rất oke rồi.

#### 4.4. 'home_country'

Cột này thì nên check unique sẽ ổn nhất.

In [417]:
bank_churned_optimized.select(
    col('home_country')
).distinct().show()

+------------+
|home_country|
+------------+
|       Spain|
|     Unknown|
|      France|
|     Germany|
+------------+



4 giá trị rất đáng yêu rồi!

#### 4.5. 'gender'

Từ cột này thì cột nào có vấn đề mới có nói gì tiếp á không thì thôi.

In [425]:
bank_churned_optimized.select(
    col('gender')
).distinct().show()

+------+
|gender|
+------+
|Female|
|  Male|
+------+



#### 4.6. 'age'

In [429]:
bank_churned_optimized.orderBy('age', ascending = True).show()

+-----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|   15641688|         644|       Spain|  Male| 18|           7|         0.0|               1|            0|       1| 59645.24|        1|
|   15665521|         642|     Germany|  Male| 18|           5|   111183.53|               2|            0|       1| 10063.75|        0|
|   15673180|         727|     Germany|Female| 18|           2|     93816.7|               2|            1|       0|126172.11|        0|
|   15787619|         844|      France|  Male| 18|           2|   160980.03|               1|            0|       0|145936.28|        0|
|   15757821|         771|       Spain|  

In [431]:
bank_churned_optimized.orderBy('age', ascending = False).show()

+-----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|   15764927|         753|      France|  Male| 92|           3|   121513.31|               1|            0|       1|195563.99|        0|
|   15660878|         705|      France|  Male| 92|           1|   126076.24|               2|            1|       1| 34436.83|        0|
|   15813303|         513|       Spain|  Male| 88|          10|         0.0|               2|            1|       1| 52952.24|        0|
|   15578006|         787|      France|Female| 85|          10|         0.0|               2|            1|       1|116537.96|        0|
|   15798024|         537|     Germany|  

#### 4.6. 'years_stayed'

In [436]:
bank_churned_optimized.orderBy('years_stayed', ascending = True).show()

+-----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|   15801265|         689|       Spain|Female| 45|           0|    57784.22|               1|            1|       0| 197804.0|        1|
|   15709523|         525|     Germany|Female| 30|           0|   157989.21|               2|            1|       1|100687.67|        0|
|   15635078|         714|       Spain|  Male| 45|           0|   124693.48|               1|            0|       1|187194.15|        0|
|   15658929|         683|       Spain|  Male| 29|           0|   133702.89|               1|            1|       0| 55582.54|        1|
|   15782659|         527|      France|  

In [438]:
bank_churned_optimized.orderBy('years_stayed', ascending = False).show()

+-----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|customer_id|credit_score|home_country|gender|age|years_stayed|bank_balance|no_products_used|hasCreditCard|isActive|   salary|isChurned|
+-----------+------------+------------+------+---+------------+------------+----------------+-------------+--------+---------+---------+
|   15747724|         671|       Spain|Female| 34|          10|         0.0|               1|            1|       0| 23235.38|        0|
|   15598161|         654|      France|  Male| 47|          10|         0.0|               2|            1|       0|170481.98|        0|
|   15744582|         680|      France|Female| 24|          10|         0.0|               3|            1|       0|154971.63|        1|
|   15760085|         684|     Germany|Female| 48|          10|   126384.42|               1|            1|       1|198129.36|        0|
|   15595388|         594|      France|Fe